<a href="https://colab.research.google.com/github/beniguha/GenAI/blob/main/Agentic-AI-Experiments/wikipedia_with_gemini_pro_model_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain-google-genai
!pip install wikipedia
!pip install --upgrade langchain langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=6ee77ca2e08a10f2c0d95822aa6804615d16add4bebfec345dc81c3145bd3785
  Stored in directory: /root/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.8/495.8 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
  

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
#from langchain.agents import load_tools, initialize_agent
import getpass
import os
from langchain import agents
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults  # Example alternative tool
from langchain_classic.agents import AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.messages import HumanMessage
from langchain.agents import create_agent

In [3]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=GOOGLE_API_KEY)
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

# Example 2: Math Tool (requires a Python REPL or similar executor in the agent)
# Often the agent framework handles basic math itself, or you use a specific executor.
# For simple LLM math, you might just rely on the LLM's own capabilities or a Python agent.

# Collect your tools into a list
tools = [wikipedia_tool]

# The create_react_agent function is now part of the `langchain.agents` module and uses an AgentExecutor


# Define the prompt for the ReAct agent
prompt = PromptTemplate.from_template("""
Answer the following questions as best you can. You have access to the following tools:
{tools}
Question: {input}""")

# Create the agent
# Note: create_react_agent often needs a specific *type* of prompt, not just any PromptTemplate
# The error indicates that create_react_agent takes 2 positional arguments but 3 were given.
# This means the `prompt` argument should not be directly passed here.

# Initialize the AgentExecutor
agent_executor = create_agent(model=llm,tools=tools,system_prompt="You are a helpful assistant.")# Invoke directly (it handles the loop for you)
response = agent_executor.invoke({"messages": [{"role": "user", "content": "Who is Lin Dan"}]})
print(response["messages"][-1].content)

[{'type': 'text', 'text': 'Lin Dan (born 14 October 1983) is a Chinese former professional badminton player.', 'extras': {'signature': 'CogEAXLI2nxgOrR7O9owrc1R99DtjciZJll2r+97AafkRbLtsbLQS8KeDoPsja0JGz1k644EkdeplZdxqjGY7tx9CV2qHkvdtYQ1rdYukwoVQxtEIGn2ex6MjiYiAK7vRC1h3tUB2xbbzY/4BkMDKPaZy/OKsWLMsha7LxH4WrCaA+P1jlOUJ3YoN8gs5cIU732k3AFXWnw8llFntC7IJj7bFWARZgTwJ8TRErgm2EC8V7/dnvDlWffydRzxlWYUYspUGVGhSS9P3dy+hN03v9MmaSiVKQgsg45fk953NF1bMoG/PYfiPqL1VkG+Au/tbNYjne7334GJikqjMsB+ZPb5flOi0vW8A2EVh/bdf7K7eyefuAtk8ZTpUWlIL8KjBZiX+6WoSPPRDANhiFZnWHaim5mfh9qykNIQfN8HBxbrb1ehV81oQbpFBuihMLlqUZtGsD3OjFut0LVWiDkDCzGqEHExsH4jSL6Qw7SkXiKmN3Xb19lbwaUOce4oQE4TA7tLRwKkpLk6ibzki+lRrKWsFY4p4YzZX/ykKls3I6p81Rlpzz8y6FeATwTt3kBW+K8DPIcxPd43542upfk2pccjErZ2D1Urs4MAlDccdaM+F4b9f/QcZVcfeGy50kiuAsy4hzcZXcHhhHef+2J807gQbTGGKSpGLMkVuU60rhhWXldvsy5NxhKtwg=='}}]


In [ ]:
print(llm)